In [1]:
# https://www.youtube.com/watch?v=N_nuxl7eWg4

In [2]:
# pip install yt_dlp
# conda install -c conda-forge ffmpeg
# conda install -c conda-forge moviepy

In [3]:
## Youtube 영상 다운로드

In [4]:
import yt_dlp

def download_youtube_video_compatible(url, output_filename="video.mp4"):
    ydl_opts = {
        'format': 'bestvideo[vcodec^=avc1]+bestaudio[acodec^=mp4a]/mp4',
        'outtmpl': output_filename,
        'merge_output_format': 'mp4',
        'noplaylist': True,
        'quiet':False
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        print(f"🎬 다운로드 시작: {url}")
        ydl.download([url])
        print(f"✅ 다운로드 완료: '{output_filename}' 으로 저장됨")

youtube_url = "https://www.youtube.com/watch?v=N_nuxl7eWg4"
video_file = "./data/dino.mp4"

# 실행
download_youtube_video_compatible(youtube_url, video_file)

Deprecated Feature: Support for Python version 3.8 has been deprecated. Please update to Python 3.9 or above


🎬 다운로드 시작: https://www.youtube.com/watch?v=N_nuxl7eWg4
[youtube] Extracting URL: https://www.youtube.com/watch?v=N_nuxl7eWg4
[youtube] N_nuxl7eWg4: Downloading webpage
[youtube] N_nuxl7eWg4: Downloading ios player API JSON
[youtube] N_nuxl7eWg4: Downloading mweb player API JSON
[youtube] N_nuxl7eWg4: Downloading player fc2a56a5


         player = https://www.youtube.com/s/player/fc2a56a5/player_ias.vflset/en_US/base.js
         n = FD0PghUQB3jzkMn ; player = https://www.youtube.com/s/player/fc2a56a5/player_ias.vflset/en_US/base.js
         player = https://www.youtube.com/s/player/fc2a56a5/player_ias.vflset/en_US/base.js
         n = Qesr2M8Ya5wcCtg ; player = https://www.youtube.com/s/player/fc2a56a5/player_ias.vflset/en_US/base.js


[youtube] N_nuxl7eWg4: Downloading m3u8 information
[info] N_nuxl7eWg4: Downloading 1 format(s): 137+140
[download] ./data/dino.mp4 has already been downloaded
✅ 다운로드 완료: './data/dino.mp4' 으로 저장됨


In [5]:
## mp4파일에서 음성 추출
from moviepy.editor import VideoFileClip

def extract_audio_from_mp4(mp4_path, mp3_path):
    
    print(f"🎧 오디오 추출 시작: {mp4_path}")
    # MP4 비디오 파일 로드
    video = VideoFileClip(mp4_path)
    # 오디오 트랙 추출
    audio = video.audio
    # 오디오를 MP3 파일로 저장
    audio.write_audiofile(mp3_path)
    # 리소스 정리
    audio.close()
    video.close()
    print(f"✅ 오디오 추출 완료: '{mp3_path}' 으로 저장됨")

video_file = "./data/dino.mp4"
audio_file = "./data/dino.mp3"

# 사용 예시
extract_audio_from_mp4(video_file, audio_file)

🎧 오디오 추출 시작: ./data/dino.mp4
MoviePy - Writing audio in ./data/dino.mp3


MoviePy - Done.
✅ 오디오 추출 완료: './data/dino.mp3' 으로 저장됨


In [6]:
from openai import OpenAI

# whisper model
# https://github.com/openai/whisper

openai_key = ""
client = OpenAI(api_key=openai_key)

audio_file = open(audio_file, "rb")

transcription = client.audio.transcriptions.create(
  file=audio_file,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["segment"] # segment
)

In [7]:
# whisper 결과
segments = transcription.segments

for seg in segments[:10]:
    
    item = eval(seg.json())

    start_time = item['start']
    end_time = item['end']
    text = item['text']
    
    print(f"{start_time} ~ {end_time} : {text}")
    print('-'*10)

0.0 ~ 2.799999952316284 :  갑자기 영감이 딱! 하고 떠올라서
----------
2.799999952316284 ~ 4.400000095367432 :  어떻게 한 번 얘기해볼까요?
----------
5.300000190734863 ~ 6.599999904632568 :  뭐가 많은 건가요?
----------
13.100000381469727 ~ 14.699999809265137 :  안녕하세요, 안녕하세요
----------
15.0 ~ 16.5 :  또 다른 분들 안 계신가?
----------
17.399999618530273 ~ 18.700000762939453 :  안녕하세요, 안녕하세요
----------
18.700000762939453 ~ 19.200000762939453 :  이경!
----------
19.200000762939453 ~ 19.899999618530273 :  안녕하세요
----------
20.100000381469727 ~ 20.700000762939453 :  어머, 어머
----------
20.700000762939453 ~ 21.799999237060547 :  너무 귀여워요
----------


In [8]:
import json

def format_segments_for_prompt(segments):
    
    formatted = []
    
    for seg in segments:

        seg = eval(seg.json()) # text -> json
        
        start = round(seg["start"], 1)
        end = round(seg["end"], 1)
        text = seg["text"].strip().replace("\n", " ")
        
        item = {
            "start" : start,
            "end" : end,
            "text" : text
        }
        formatted.append(item)
        segments_str = json.dumps(formatted, ensure_ascii=False, indent=2)
        
    return segments_str

processed_segments = format_segments_for_prompt(segments)

In [9]:
print(processed_segments)

[
  {
    "start": 0.0,
    "end": 2.8,
    "text": "갑자기 영감이 딱! 하고 떠올라서"
  },
  {
    "start": 2.8,
    "end": 4.4,
    "text": "어떻게 한 번 얘기해볼까요?"
  },
  {
    "start": 5.3,
    "end": 6.6,
    "text": "뭐가 많은 건가요?"
  },
  {
    "start": 13.1,
    "end": 14.7,
    "text": "안녕하세요, 안녕하세요"
  },
  {
    "start": 15.0,
    "end": 16.5,
    "text": "또 다른 분들 안 계신가?"
  },
  {
    "start": 17.4,
    "end": 18.7,
    "text": "안녕하세요, 안녕하세요"
  },
  {
    "start": 18.7,
    "end": 19.2,
    "text": "이경!"
  },
  {
    "start": 19.2,
    "end": 19.9,
    "text": "안녕하세요"
  },
  {
    "start": 20.1,
    "end": 20.7,
    "text": "어머, 어머"
  },
  {
    "start": 20.7,
    "end": 21.8,
    "text": "너무 귀여워요"
  },
  {
    "start": 21.8,
    "end": 22.9,
    "text": "무슨 일이야"
  },
  {
    "start": 23.1,
    "end": 23.7,
    "text": "안녕하세요"
  },
  {
    "start": 24.3,
    "end": 24.8,
    "text": "어머"
  },
  {
    "start": 25.3,
    "end": 27.2,
    "text": "이 귀여운 쿼카 인형 안고 있어도 되나요?"
  },
  {
    "start": 27.4,
   

In [10]:
prompt = """

다음은 영상에서 추출한 여러 문장(세그먼트)입니다. 

이 중에서 시청자의 흥미를 끌 수 있는, 중요한 정보나 감정적으로 인상적인 문장이 나와있는 구간을 골라서 유튜브 클립으로 만드려고 합니다.
이 영상은 은행 상품 소개 영상이기 때문에 고객에게 유용한 혜택이나 정보를 알려줄 수 있는 내용이면 더 좋을 것 같습니다.

위의 내용을 만족하는 세그먼트를 포함하여 시작, 끝시간이 약 30초가 될 수 있게 적절하게 세그먼트를 합쳐서 이를 알려주세요.

The output should be in JSON format (without codeblock ```.)


형식: 
{{
    {{ "start": [시작 시간], "end": [끝 시간], "text": "[문장 내용]" }}
}}

세그먼트:
{processed_segments}

"""

prompt = prompt.format(processed_segments=processed_segments)

In [11]:
response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
)

In [12]:
print(response.output_text)

{
    "start": 67.3,
    "end": 96.6,
    "text": "궁금한 적금은 31일짜리 적금인데요 매일매일 하루에 한 번 입금할 때마다 랜덤으로 5대 금리가 쌓이고 있어요 금리는 최대 연 7.5%까지 드리고 있습니다 7.5%? 7.5, 진짜 높아요 그러면 금리가 연 7.5%인데 매일매일 랜덤 금리로 조금씩 조금씩 쌓이는 건가요? 맞아요, 이게 어떤 금리가 나올지도 모르고 다이노탱의 귀여운 스토리들도 매일 나오는 게 재미 포인트예요 경준님 오늘 약간 MC 같은데요? 저... 보고 그렇게 하라고 하더라고요"
}


In [13]:
json_str = response.output_text
parsed = json.loads(json_str)
print(parsed)

{'start': 67.3, 'end': 96.6, 'text': '궁금한 적금은 31일짜리 적금인데요 매일매일 하루에 한 번 입금할 때마다 랜덤으로 5대 금리가 쌓이고 있어요 금리는 최대 연 7.5%까지 드리고 있습니다 7.5%? 7.5, 진짜 높아요 그러면 금리가 연 7.5%인데 매일매일 랜덤 금리로 조금씩 조금씩 쌓이는 건가요? 맞아요, 이게 어떤 금리가 나올지도 모르고 다이노탱의 귀여운 스토리들도 매일 나오는 게 재미 포인트예요 경준님 오늘 약간 MC 같은데요? 저... 보고 그렇게 하라고 하더라고요'}


In [14]:
from moviepy.editor import VideoFileClip
import os

def cut_segments_from_video(video_path,                             
                            segments, 
                            output_dir="clips"):
    
    os.makedirs(output_dir, exist_ok=True)

    video = VideoFileClip(video_path)

    for i, seg in enumerate(segments):
        start = seg["start"]
        end = seg["end"]
        text = seg["text"][:30].replace(" ", "_").replace("?", "").replace(".", "")  # 짧은 파일명용

        output_path = os.path.join(output_dir, f"clip_{i+1}_{text}.mp4")

        print(f"🔪 자르는 중: {start}초 ~ {end}초 -> {output_path}")

        clip = video.subclip(start, end)
        clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

    print(f"✅ 총 {len(segments)}개의 클립이 저장되었습니다. 폴더: {output_dir}")

In [15]:
segments_to_cut = [parsed]

clip_title = response.output_text
clip_path = f"./{clip_title}"

cut_segments_from_video(video_file, segments_to_cut)

🔪 자르는 중: 67.3초 ~ 96.6초 -> clips/clip_1_궁금한_적금은_31일짜리_적금인데요_매일매일_하루에_한.mp4
Moviepy - Building video clips/clip_1_궁금한_적금은_31일짜리_적금인데요_매일매일_하루에_한.mp4.
MoviePy - Writing audio in clip_1_궁금한_적금은_31일짜리_적금인데요_매일매일_하루에_한TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video clips/clip_1_궁금한_적금은_31일짜리_적금인데요_매일매일_하루에_한.mp4



Moviepy - Done !
Moviepy - video ready clips/clip_1_궁금한_적금은_31일짜리_적금인데요_매일매일_하루에_한.mp4
✅ 총 1개의 클립이 저장되었습니다. 폴더: clips


In [16]:
## 타이틀 추천

clip_title_prompt = """

아래 내용은 동영상 대본이야. 이 대본으로 클립 제목을 만드려고 하는데 추천 해줄래?
추가 설명없이 클립 제목만 추천해줘.

대본:
{content}

"""

content = parsed['text']

request = clip_title_prompt.format(content=content)

response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": request,
        }
    ],
)

print(response.output_text)

"31일간의 랜덤 금리 모험: 최대 연 7.5%의 매력"
